In [1]:
import sys, time, pickle, torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
sys.path.insert(0, '../../Preprocess')
import numpy as np
import pandas as pd
from preload_models import get_sst2_tok_n_model
from _utils import sample_random_glue_sst2, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info

In [2]:
sst2_data_raw, targets, idxs = sample_random_glue_sst2()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3b24abff24d1d8c0.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5960909ab3834668.arrow


In [3]:
tokenizer, model = get_sst2_tok_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import DeepLift
from captum.attr import visualization 

In [6]:
# lime = Lime(forward_func=model.forward)
deep_lift = DeepLift(model)

In [7]:
def generate_record(raw_review, target):
    #tokenizer operations
    tokenized = tokenizer(raw_review, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokenized['offset_mapping']
    conti_map = get_continuation_mapping(offset_mapping)
    input_ids = torch.tensor(tokenized['input_ids']).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    
    #categorizing results
    pred_class = 'Pos' if pred_prob > 0.5 else 'Neg' 
    true_class = 'Pos' if target > 0.5 else 'Neg' 
    
    #attribution algorithm working
    attribution, delta = deep_lift.attribute(input_emb, return_convergence_delta=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = delta
    
    
    #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(sst2_data_raw, targets), start=1):
    print(f'Raw review: {datum_raw}')
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))
   

Raw review: its oscar nomination 
GT target: 1


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/deep_lift.py:322: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(


word attr tensor([-0.9534,  0.0890, -0.0922,  0.2486,  0.0072, -0.0956, -0.0596],
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.9534, grad_fn=<UnbindBackward0>), tensor(0.0890, grad_fn=<UnbindBackward0>), tensor(0.1564, grad_fn=<AddBackward0>), tensor(0.0072, grad_fn=<UnbindBackward0>), tensor(-0.0956, grad_fn=<UnbindBackward0>), tensor(-0.0596, grad_fn=<UnbindBackward0>)]
detokenized ['<s>', 'its', 'o', 'scar', 'nomination', '', '</s>']
len conti_raw 6
conti_raw ['<s>', 'its', 'oscar', 'nomination', '', '</s>']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Pos (1.00),Neg,-0.86,#s its oscar nomination #/s


<IPython.core.display.HTML object>
Raw review: shenanigans and slapstick 
GT target: 1
word attr tensor([-0.4782, -0.0438, -0.0442, -0.3404, -0.0757, -0.2553, -0.0311,  0.7514,
        -0.0623,  0.1061], grad_fn=<DivBackward0>)
conti attr [tensor(-0.4782, grad_fn=<UnbindBackward0>), tensor(-0.5041, grad_fn=<AddBackward0>), tensor(-0.2553, grad_fn=<UnbindBackward0>), tensor(0.7203, grad_fn=<AddBackward0>), tensor(-0.0623, grad_fn=<UnbindBackward0>), tensor(0.1061, grad_fn=<UnbindBackward0>)]
detokenized ['<s>', 's', 'hen', 'an', 'igans', 'and', 'slap', 'stick', '', '</s>']
len conti_raw 6
conti_raw ['<s>', 'shenanigans', 'and', 'slapstick', '', '</s>']


...

<IPython.core.display.HTML object>
Raw review: unrewarding 
GT target: 0
word attr tensor([-0.1385,  0.0288, -0.2694,  0.8694, -0.2586,  0.0291,  0.2895],
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.1385, grad_fn=<UnbindBackward0>), tensor(0.3702, grad_fn=<AddBackward0>), tensor(0.0291, grad_fn=<UnbindBackward0>), tensor(0.2895, grad_fn=<UnbindBackward0>)]
detokenized ['<s>', 'un', 're', 'ward', 'ing', '', '</s>']
len conti_raw 4
conti_raw ['<s>', 'unrewarding', '', '</s>']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.00),Pos,0.55,#s unrewarding #/s


<IPython.core.display.HTML object>
Raw review: old-hat 
GT target: 0
word attr tensor([-0.5848, -0.3870,  0.3062,  0.2848,  0.5519, -0.1697],
       grad_fn=<DivBackward0>)
conti attr [tensor(-0.5848, grad_fn=<UnbindBackward0>), tensor(0.2041, grad_fn=<AddBackward0>), tensor(0.5519, grad_fn=<UnbindBackward0>), tensor(-0.1697, grad_fn=<UnbindBackward0>)]
detokenized ['<s>', 'old', '-', 'hat', '', '</s>']
len conti_raw 4
conti_raw ['<s>', 'old-hat', '', '</s>']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.00),Neg,0.00,#s old-hat #/s


<IPython.core.display.HTML object>


In [9]:
save_info(idxs, sst2_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='deeplift_out.pkl')

{'indices': [27798,
  28244,
  48083,
  8795,
  91,
  20682,
  59890,
  41763,
  19795,
  9957,
  28334,
  55664,
  16438,
  18031,
  9297,
  66913,
  9289,
  6789,
  25294,
  33294,
  29699,
  41960,
  6554,
  8772,
  17921,
  8027,
  49091,
  54186,
  15080,
  22382,
  23856,
  9464,
  23831,
  59434,
  17862,
  24584,
  26216,
  63094,
  55787,
  3993,
  25463,
  17540,
  51128,
  46224,
  32656,
  30105,
  28646,
  17011,
  7812,
  48236],
 'raw_data': ['its oscar nomination ',
  'shenanigans and slapstick ',
  'an unsettling sight , ',
  'the climactic hourlong cricket match ',
  'alternating between facetious comic parody and pulp melodrama , this smart-aleck movie ... tosses around some intriguing questions about the difference between human and android life ',
  'to be a part of that elusive adult world ',
  'emotional power ',
  'reminds you of why animation is such a perfect medium for children , because of the way it allows the mind to enter and accept another world ',
  'un

In [10]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, RobertaTokenizer, RobertaForTokenClassification
from transformers import pipeline
import torch

In [11]:
model_name_or_path = "siebert/sentiment-roberta-large-english" 
tokenizer = RobertaTokenizer.from_pretrained(model_name_or_path)
model = RobertaForTokenClassification.from_pretrained(model_name_or_path)  # Pytorch
# model = TFAutoModelForTokenClassification.from_pretrained(model_name_or_path)  # Tensorflow


Some weights of the model checkpoint at siebert/sentiment-roberta-large-english were not used when initializing RobertaForTokenClassification: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [12]:

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = 'hi was AMAZING'


In [13]:
out = tokenizer(example)
input_ids = out['input_ids']

In [14]:
toked = [w.replace('Ġ', '') for w in tokenizer.convert_ids_to_tokens(input_ids)]
print(toked)

['<s>', 'hi', 'was', 'AM', 'AZ', 'ING', '</s>']
